In [1]:
# Enable TF Eager execution at the beginning  ==> for decoder later
import tensorflow as tf
tfe = tf.contrib.eager
tfe.enable_eager_execution()
Modes = tf.estimator.ModeKeys


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [ ]:
# %%bash
# DATA_DIR=./data
# TMP_DIR=$DATA_DIR/tmp
# rm -rf $DATA_DIR $TMP_DIR
# mkdir -p $DATA_DIR $TMP_DIR
# # Generate data
# t2t-datagen \
#   --t2t_usr_dir=./train \
#   --problem="onlinereview" \
#   --data_dir=$DATA_DIR \
#   --tmp_dir=$TMP_DIR

In [ ]:
# 3637767 Examples

In [2]:
from train import problem
problem.onlinereview('onlinereview')

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### Confirm data is correct

In [3]:
from tensor2tensor.utils import registry
from tensor2tensor import problems

DATA_LOC = './data' 
t2t_problem = problems.problem('onlinereview')
vocab_name = "./data/vocab.onlinereview.32768.subwords"

# Get the encoders from the problem
encoders = t2t_problem.feature_encoders(DATA_LOC)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
    inputs = encoders["inputs"].encode(input_str) + [1]  
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  
    return {"inputs": batch_inputs}

def decode(integers):
    integers = list(np.squeeze(integers))
    if 1 in integers:
        integers = integers[:integers.index(1)]
    return encoders["inputs"].decode(np.squeeze(integers))

In [4]:
encoders = t2t_problem.feature_encoders(DATA_LOC)
encoders

{'inputs': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder at 0x7f6a06cb26a0>,
 'targets': <tensor2tensor.data_generators.text_encoder.SubwordTextEncoder at 0x7f6a06cb26a0>}

In [5]:
# ende_problem.generate_data(data_dir, tmp_dir)
example = tfe.Iterator(t2t_problem.dataset(Modes.TRAIN, DATA_LOC)).next()
inputs = [int(x) for x in example["inputs"].numpy()] # Cast to ints.
targets = [int(x) for x in example["targets"].numpy()] # Cast to ints.

INFO:tensorflow:Reading data files from ./data/onlinereview-train*
INFO:tensorflow:partition: 0 num_data_files: 100
Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


In [6]:
import numpy as np
# Example inputs as int-tensor.
print("Inputs, encoded:")
print(inputs)
print("Inputs, decoded:")
# Example inputs as a sentence.
print(decode(inputs))

print("---"*10)
# Example targets as int-tensor.
print("Targets, encoded:")
print(targets)
# Example targets as a sentence.
print("Targets, decoded:")
print(decode(targets))

Inputs, encoded:
[61, 4397, 22997, 3, 14, 10, 2, 1052, 29065, 9, 14, 39953, 29364, 448, 32221, 1150, 5152, 215, 112, 2582, 91, 447, 33, 346, 16, 7, 3170, 804, 12, 2, 2405, 39, 451, 51, 6, 69, 27, 14411, 9573, 12018, 2263, 21, 5, 1019, 3, 1019, 673, 9, 14, 6713, 9, 22653, 8, 51, 883, 2, 643, 1130, 91, 34074, 9485, 1294, 78, 10, 353, 5, 4060, 235, 124, 76, 111, 35169, 2946, 3, 10737, 4, 12372, 1190, 9, 5826, 6479, 4, 22228, 28, 19974, 1312, 8, 7, 95, 30294, 277, 523, 43, 14, 94, 35, 1084, 58, 7977, 604, 121, 1186, 3, 2742, 58, 6, 28188, 5203, 6, 27, 6666, 1088, 91, 45, 959, 4, 89, 8770, 16807, 158, 12, 339, 6, 20937, 12353, 1984, 972, 2, 1186, 16, 7, 131, 8, 12, 333, 3, 11, 1902, 58, 481, 5, 20938, 3, 77, 10, 55, 7, 131, 6, 31, 13, 7, 95, 84, 34074, 1880, 18681, 32132, 19, 6, 284, 412, 3, 77, 10, 55, 7, 24, 5, 12721, 20938, 131, 8, 14, 179, 1902, 58, 972, 51, 2, 360, 9, 1186, 16, 14033, 45, 58, 3, 77, 6064, 58, 98, 5, 873, 9, 31510, 848, 16, 7, 25425, 91, 255, 437, 193, 1099, 853, 1872, 

## Maxlen 256 -one more ing

In [12]:
from tensor2tensor import problems
from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json

MODEL = 'transformer'
TRAIN_DIR = 'model_files_base256' #s-v2
DATA_LOC = './data'
PROBLEM = 'onlinereview'

# Define hparams
HPARAMS = 'transformer_base'
hparams = create_hparams(HPARAMS)
# hparams.prepend_mode = 'prepend_inputs_full_attention'
hparams.batch_size = 1024 #2048 #512 
hparams.learning_rate_warmup_steps = 6000
hparams.learning_rate = 0.1 
# hparams.num_encoder_layers = 6
hparams.num_hidden_layers = 6
# hparams.num_decoder_layers = 6
hparams.dropout =0.2
# hparams.num_heads =16
hparams.attention_dropout = 0.2 ##
hparams.max_length = 256
# hparams.max_input_seq_length = 512

decode_hp = decoding.decode_hparams()
# decode_hp.block_size =6
decode_hp.return_beams = True #True
decode_hp.beam_size = 2

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))


{'learning_rate_cosine_cycle_steps': 250000, 'optimizer_momentum_nesterov': False, 'tpu_enable_host_call': False, 'label_smoothing': 0.1, 'hidden_size': 512, 'nbr_decoder_problems': 1, 'heads_share_relative_embedding': False, 'moe_overhead_eval': 2.0, 'optimizer_multistep_accumulate_steps': None, 'video_num_input_frames': 1, 'prepend_mode': 'none', 'learning_rate_decay_steps': 5000, 'pad_batch': False, 'optimizer_adafactor_beta1': 0.0, 'learning_rate_warmup_steps': 6000, 'split_to_length': 0, 'parameter_attention_key_channels': 0, 'attention_value_channels': 0, 'shared_embedding': False, 'filter_size': 2048, 'shared_embedding_and_softmax_weights': True, 'optimizer_adam_beta2': 0.997, 'multiproblem_fixed_train_length': -1, 'causal_decoder_self_attention': True, 'batch_shuffle_size': 512, 'overload_eval_metric_name': '', 'layer_preprocess_sequence': 'n', 'attention_variables_3d': False, 'unidirectional_encoder': False, 'no_data_parallelism': False, 'summarize_grads': False, 'optimizer': 

In [9]:
import timeit
# running time calculation
start = timeit.default_timer()

t2t_problem = problems.problem(PROBLEM)

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=2000,
    )

t2t_model_base256 = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        problem_name=t2t_problem,
        model_name=MODEL,
        data_dir=DATA_LOC, 
        train_steps=40000,
        eval_steps=200,
        min_eval_frequency=2000,
        eval_use_test_set=True,
    )

# hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_base256.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_environment': 'local', '_model_dir': 'model_files_base256', '_eval_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_device_fn': None, '_num_worker_replicas': 0, '_log_step_count_steps': 100, 't2t_device_info': {'num_async_replicas': 1}, '_master': '', '_task_type': None, '_task_id': 0, 'use_tpu': False, '_num_ps_replicas': 0, '_save_checkpoints_steps': 2000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f95c67eb358>, '_train_distribute': None, '_keep_checkpoint_max': 20, '_save_checkpoints_secs': None, '_protocol': None, '_is_chief': True, '_session_config': gpu_options {
  per_process_gpu_

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


INFO:tensorflow:partition: 0 num_data_files: 100


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-05T18:28:51Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_base256/model.ckpt-34000
INFO:tensorflow:Running local_init_op.
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-05T18:37:25Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_base256/model.ckpt-36000
INFO:tensorflow:Running local_init_op.
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-05T18:45:55Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_base256/model.ckpt-38000
INFO:tensorflow:Running local_init_op.
INFO:tensor

In [13]:
import timeit
# running time calculation
start = timeit.default_timer()

t2t_problem = problems.problem(PROBLEM)

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=2000,
    )

t2t_model_base256 = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        problem_name=t2t_problem,
        model_name=MODEL,
        data_dir=DATA_LOC, 
        train_steps=50001,
        eval_steps=200,
        min_eval_frequency=2000,
        eval_use_test_set=True,
    )

# hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_base256.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_environment': 'local', '_model_dir': 'model_files_base256', '_eval_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_device_fn': None, '_num_worker_replicas': 0, '_log_step_count_steps': 100, 't2t_device_info': {'num_async_replicas': 1}, '_master': '', '_task_type': None, '_task_id': 0, 'use_tpu': False, '_num_ps_replicas': 0, '_save_checkpoints_steps': 2000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f956c780908>, '_train_distribute': None, '_keep_checkpoint_max': 20, '_save_checkpoints_secs': None, '_protocol': None, '_is_chief': True, '_session_config': gpu_options {
  per_process_gpu_

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


INFO:tensorflow:Reading data files from ./data/onlinereview-train*
INFO:tensorflow:partition: 0 num_data_files: 100


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Base learning rate: 2.000000
INFO:tensorflow:Trainable Variables Total size: 64628224
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-06T02:32:03Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_base256/model.ckpt-44000
INFO:tensorflow:Running local_init_op.
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-06T02:40:37Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_base256/model.ckpt-46000
INFO:tensorflow:Running local_init_op.
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-06T02:49:07Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_files_base256/model.ckpt-48000
INFO:tensorflow:Running local_init_op.
INFO:tensor

### - decoding check-

In [16]:
from tensor2tensor import problems
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor.utils.trainer_lib import create_hparams
import json
import numpy as np

DATA_LOC = './data' 
t2t_problem = problems.problem('onlinereview')
vocab_name = "./data/vocab.onlinereview.32768.subwords"

# Get the encoders from the problem
encoders = t2t_problem.feature_encoders(DATA_LOC)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
    inputs = encoders["inputs"].encode(input_str) + [1]  
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  
    return {"inputs": batch_inputs}

def decode(integers):
    integers = list(np.squeeze(integers))
    if 1 in integers:
        integers = integers[:integers.index(1)]
    return encoders["inputs"].decode(np.squeeze(integers))

MODEL = 'transformer'
HPARAMS = 'transformer_base'
hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_LOC, problem_name=t2t_problem)
# hparams.prepend_mode = 'prepend_inputs_full_attention'

model = registry.model(MODEL)(hparams, Modes.EVAL) #tensorflow_exp_fn

ckpt_path = tf.train.latest_checkpoint('model_file_base256') #model.ckpt
print(ckpt_path)

#Restore and summarize
def summarize(inputs):
    encoded_inputs = encode(inputs)
    with tfe.restore_variables_on_create(ckpt_path):
        model_output = model.infer(encoded_inputs)["outputs"]
    return decode(model_output)


model_file_base256/model.ckpt-40000


In [23]:
review1 = '''
we have many of the old, old issue. but the number had depleted. there were not  enough books to allow us to use them regularly. with the additional supply the books will be used more often.  they arre a good old standby for gospel singing.'''

gold1 = '''i was disappointed that you would only allow me to purchase 4 when your inventory showed that you had 14 available'''

review2 = '''
my daughter wore this every other day for maybe half an hour at most the waistband has completely separated from the elastic after two weeks of this light use
'''
gold2 = '''this was great until it fell apart two weeks later'''

# review3 = '''
# unfortunately this skirts elastic is not stretchy enough to accommodate the 29 year old range , this is crazy i would say it only stretches through about 1 size i would throw this in a costume play chest with a safety pin handy and call it a day  
# '''
# gold3 = '''i should have paid more attention to the age range'''

for review, summary in [(review1,gold1), (review2, gold2), (review3, gold3)]:
    pred_summary = summarize(review)
    print("Review: %s" % review)
    print("Pred_summary: %s" % pred_summary)
    print("Gold_summary:%s" % summary)
    print('-----'*10, '\n')


Review: 
we have many of the old, old issue. but the number had depleted. there were not  enough books to allow us to use them regularly. with the additional supply the books will be used more often.  they arre a good old standby for gospel singing.
Pred_summary: i have been a great for a few months of the price
Gold_summary:i was disappointed that you would only allow me to purchase 4 when your inventory showed that you had 14 available
-------------------------------------------------- 

Review: 
my daughter wore this every other day for maybe half an hour at most the waistband has completely separated from the elastic after two weeks of this light use

Pred_summary: i am not a little book for the price
Gold_summary:this was great until it fell apart two weeks later
-------------------------------------------------- 

Review: 
unfortunately this skirts elastic is not stretchy enough to accommodate the 29 year old range , this is crazy i would say it only stretches through about 1 siz

## Base -512

In [3]:
from tensor2tensor import problems
from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json

MODEL = 'transformer'
TRAIN_DIR = 'model_file_base512'
DATA_LOC = './data'
PROBLEM = 'onlinereview'

# Define hparams
HPARAMS = 'transformer_base'
hparams = create_hparams(HPARAMS)
# hparams.prepend_mode = 'prepend_inputs_full_attention'
hparams.batch_size = 1024 #2048 #2048 #512 
hparams.learning_rate = 0.1 
# hparams.num_encoder_layers = 6
hparams.num_hidden_layers = 6
# hparams.num_decoder_layers = 6
hparams.dropout =0.1
# hparams.num_heads =16
hparams.attention_dropout = 0.1 ##
hparams.max_length = 512
# hparams.max_input_seq_length = 512

decode_hp = decoding.decode_hparams()
decode_hp.return_beams = True
decode_hp.beam_size = 4

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))


{'split_to_length': 0, 'filter_size': 2048, 'attention_dropout_broadcast_dims': '', 'optimizer_adafactor_memory_exponent': 0.8, 'video_num_target_frames': 1, 'moe_k': 2, 'num_heads': 8, 'causal_decoder_self_attention': True, 'learning_rate_decay_rate': 1.0, 'layer_prepostprocess_dropout': 0.1, 'relu_dropout': 0.1, 'heads_share_relative_embedding': False, 'hidden_size': 512, 'norm_type': 'layer', 'optimizer': 'Adam', 'factored_logits': False, 'symbol_modality_num_shards': 16, 'learning_rate_warmup_steps': 8000, 'warm_start_from_second': '', 'kernel_height': 3, 'multiproblem_reweight_label_loss': False, 'pos': 'timing', 'multiproblem_per_task_threshold': '', 'ffn_layer': 'dense_relu_dense', 'optimizer_adafactor_multiply_by_parameter_scale': True, 'self_attention_type': 'dot_product', 'moe_hidden_sizes': '2048', 'add_relative_to_values': False, 'attention_variables_3d': False, 'max_input_seq_length': 0, 'force_full_predict': False, 'parameter_attention_value_channels': 0, 'learning_rate_d

In [4]:
import timeit
# running time calculation
start = timeit.default_timer()

t2t_problem = problems.problem(PROBLEM)

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=2000,
    )

t2t_model_base = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        problem_name=t2t_problem,
        model_name=MODEL,
        data_dir=DATA_LOC, 
        train_steps=40000,
        eval_steps=200,
        min_eval_frequency=2000,
        eval_use_test_set=True,
    )

# hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_base.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_train_distribute': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 100, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7f34fa435b70>, '_save_checkpoints_secs': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f34fa435ba8>, '_num_worker_replicas': 0, '_model_dir': 'model_file_base512', '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_leve

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-04T06:12:16Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_file_base512/model.ckpt-34000
INFO:tensorflow:Running local_init_op.
INFO:tensorf

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-04T06:25:48Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_file_base512/model.ckpt-36000
INFO:tensorflow:Running local_init_op.
INFO:tensorf

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-04T06:39:17Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_file_base512/model.ckpt-38000
INFO:tensorflow:Running local_init_op.
INFO:tensorf

In [4]:
import timeit
# running time calculation
start = timeit.default_timer()

t2t_problem = problems.problem(PROBLEM)

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=2000,
    )

t2t_model_base = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        problem_name=t2t_problem,
        model_name=MODEL,
        data_dir=DATA_LOC, 
        train_steps=42001,
        eval_steps=200,
        min_eval_frequency=2000,
        eval_use_test_set=True,
    )

# hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_base.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_master': '', '_save_checkpoints_steps': 2000, '_keep_checkpoint_max': 20, '_model_dir': 'model_file_base512', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_eval_distribute': None, '_save_checkpoints_secs': None, '_device_fn': None, '_save_summary_steps': 100, '_task_type': None, '_environment': 'local', '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1f06ae6b38>, '_num_worker_replicas': 0, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism objec

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


INFO:tensorflow:partition: 0 num_data_files: 100
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Base learning rate: 2.000000
INFO:tensorflow:Trainable Variables Total size: 64628224
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown

INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from model_file_base512/model.ckpt-40000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 40000 into model_file_base512/model.ckpt.
INFO:tensorflow:Reading data files from ./data/onlinereview-test*
INFO:tensorflow:partition: 0 num_data_files: 10
INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.n

### -decoding check-

In [7]:
from tensor2tensor.utils import trainer_lib, decoding
from tensor2tensor.bin import t2t_trainer

ckpt_path = tf.train.latest_checkpoint("./model_file_base512") #model.ckpt

HPARAMS = 'transformer_base'
hparams = create_hparams(HPARAMS, data_dir=DATA_LOC, problem_name=t2t_problem)
# hparams.prepend_mode = 'prepend_inputs_full_attention'

# def create_decode_hparams():
#     decode_hp = decoding.decode_hparams()
#     decode_hp.shards = 1 #tf.flags.FLAGS.decode_shards
# #     decode_hp.shard_id = flags.FLAGS.worker_id
#     decode_in_memory = decode_hp.decode_in_memory
#     decode_hp.decode_in_memory = decode_in_memory
#     decode_hp.return_beams =True
#     decode_hp.beam_size = 2
# #     decode_hp.decode_to_file = FLAGS.decode_to_file
# #     decode_hp.decode_reference = FLAGS.decode_reference
#     return decode_hp
# decode_hp = create_decode_hparams()

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,)

estimator = trainer_lib.create_estimator(
    MODEL,
    hparams,
    RUN_CONFIG, #t2t_trainer.create_run_config(hparams),
    decode_hparams=decode_hp)

decoding.decode_interactively(estimator, hparams, decode_hp, checkpoint_path=ckpt_path)

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_train_distribute': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_save_summary_steps': 100, 'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7f3279c4c438>, '_save_checkpoints_secs': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f34bd8c3d30>, '_num_worker_replicas': 0, '_model_dir': 'model_file_base512', '_session_config': gpu_options {
  per_process_gpu_memory_fraction: 0.95
}
allow_soft_placement: true
graph_options {
  optimizer_options {
    global_jit_level: OFF
  }
}
, '_save_checkpoints_steps': 1000, '_keep_checkpoint_max': 20, '_keep_checkpoint_every_n_hou

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


INTERACTIVE MODE  num_samples=1  decode_length=100  
  it=<input_type>     ('text' or 'image' or 'label', default: text)
  ns=<num_samples>    (changes number of samples, default: 1)
  dl=<decode_length>  (changes decode length, default: 100)
  <source_string>                (decode)
  q                   (quit)
>what is there not to love about the writing of khalil gibran. i finally took the time, and not much is needed, to read the prophet.every page flows seamlessly from one to the other and his narrative writing style is infused with lessons of love, life, and more.this is a book that i will read time and time again. like many other of the greats i keep on my book shelf.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'infer'
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model_file_base512/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:BEAM 0:
INFO:tensorflow:"a must read for anyone who wants to know about the world"	Score:-11.715166
INFO:tensorflow:BEAM 1:
INFO:tensorflow:"a must read for anyone who want to know about the world"	Score:-11.719825
INFO:tensorflow:BEAM 2:
INFO:tensorflow:"a must read for anyone who want to know about the world."	Score:-11.859009
INFO:tensorflow:BEAM 3:
INFO:tensorflow:"a must read for anyone who wants to know about the world."	Score:-11.884241
INTERACTIVE MODE  num_samples=1  decode_length=100  
  it=<input_type>     ('text' or 'image' or 'label', default: text)
  ns=<num_samples>    (changes number of samples, default: 1)
  dl=<decode_length>  (changes decode length, default: 100)
  <source_string>                (decode)
  q         

In [10]:
from tensor2tensor import problems
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import registry
from tensor2tensor import models
from tensor2tensor.utils.trainer_lib import create_hparams
import json
import numpy as np

DATA_LOC = './data' 
t2t_problem = problems.problem('onlinereview')
vocab_name = "./data/vocab.onlinereview.32768.subwords"

# Get the encoders from the problem
encoders = t2t_problem.feature_encoders(DATA_LOC)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
    inputs = encoders["inputs"].encode(input_str) + [1]  
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  
    return {"inputs": batch_inputs}

def decode(integers):
    integers = list(np.squeeze(integers))
    if 1 in integers:
        integers = integers[:integers.index(1)]
    return encoders["inputs"].decode(np.squeeze(integers))

MODEL = 'transformer'
HPARAMS = 'transformer_base'
hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_LOC, problem_name=t2t_problem)
# hparams.prepend_mode = 'prepend_inputs_full_attention'

model = registry.model(MODEL)(hparams, Modes.EVAL) #tensorflow_exp_fn

ckpt_path = tf.train.latest_checkpoint("./model_file_base512/") #model.ckpt
print(ckpt_path)

#Restore and summarize
def summarize(inputs):
    encoded_inputs = encode(inputs)
    with tfe.restore_variables_on_create(ckpt_path):
        model_output = model.infer(encoded_inputs)["outputs"]
    return decode(model_output)


# inputs = '''
# we have many of the old, old issue. but the number had depleted. there were not  enough books to allow us to use them regularly. with the additional supply the books will be used more often.  they arre a good old standby for gospel singing.'''
inputs = '''
very thorough review of mksap and a great companion to the text book or online, especially if you like to review material in different formats.  there are a lot references to the tables and figures that require you to go back to the book or the online version, so it is definitely not complete. but i like the discussion. too many bad jokes, some are borderline and probably inappropriate, but a few are harmless and amusing. a classmate of mine does a great job with one of the sessions.outputs = summarize(inputs)'''
outputs = summarize(inputs)
print("Inputs: %s" % inputs)
print("Outputs: %s" % outputs)

./model_file_base512/model.ckpt-40000
Inputs: 
very thorough review of mksap and a great companion to the text book or online, especially if you like to review material in different formats.  there are a lot references to the tables and figures that require you to go back to the book or the online version, so it is definitely not complete. but i like the discussion. too many bad jokes, some are borderline and probably inappropriate, but a few are harmless and amusing. a classmate of mine does a great job with one of the sessions.outputs = summarize(inputs)
Outputs: great for the price, but not a great price


## Maxlen 1024

In [24]:
from tensor2tensor import problems
from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json

MODEL = 'transformer'
TRAIN_DIR = 'model_file_base1024'
DATA_LOC = './data'
PROBLEM = 'onlinereview'

# Define hparams
HPARAMS = 'transformer_base'
hparams = create_hparams(HPARAMS)
# hparams.prepend_mode = 'prepend_inputs_full_attention'
hparams.batch_size = 1024 #2048 #512 
hparams.learning_rate_warmup_steps = 4000
hparams.learning_rate = 0.1 
# hparams.num_encoder_layers = 6
hparams.num_hidden_layers = 6
# hparams.num_decoder_layers = 6
hparams.dropout =0.2
# hparams.num_heads =16
hparams.attention_dropout = 0.2 ##
hparams.max_length = 1024
# hparams.max_input_seq_length = 512

decode_hp = decoding.decode_hparams()
decode_hp.block_size =6
decode_hp.return_beams = True #True
decode_hp.beam_size = 2

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))


{'learning_rate_constant': 2.0, 'causal_decoder_self_attention': True, 'moe_loss_coef': 0.001, 'layer_postprocess_sequence': 'da', 'force_full_predict': False, 'multiproblem_vocab_size': -1, 'multiproblem_per_task_threshold': '', 'multiproblem_schedule_threshold': 0.5, 'batch_shuffle_size': 512, 'no_data_parallelism': False, 'video_num_target_frames': 1, 'attention_variables_3d': False, 'num_decoder_layers': 0, 'moe_hidden_sizes': '2048', 'summarize_vars': False, 'tpu_enable_host_call': False, 'summarize_grads': False, 'shared_embedding_and_softmax_weights': True, 'sampling_method': 'argmax', 'weight_dtype': 'float32', 'ffn_layer': 'dense_relu_dense', 'optimizer_multistep_accumulate_steps': None, 'weight_decay': 0.0, 'optimizer_zero_grads': False, 'heads_share_relative_embedding': False, 'optimizer_adafactor_decay_type': 'pow', 'prepend_mode': 'none', 'max_target_seq_length': 0, 'optimizer_adafactor_multiply_by_parameter_scale': True, 'multiproblem_label_weight': 0.5, 'multiproblem_fix

In [25]:
import timeit
# running time calculation
start = timeit.default_timer()

t2t_problem = problems.problem(PROBLEM)

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=2000,
    )

t2t_model_base_1024 = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        problem_name=t2t_problem,
        model_name=MODEL,
        data_dir=DATA_LOC, 
        train_steps=40001,
        eval_steps=200,
        min_eval_frequency=2000,
        eval_use_test_set=True,
    )

# hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_base_1024.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'data_parallelism': <tensor2tensor.utils.expert_utils.Parallelism object at 0x7f6953cb6e80>, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f6953cb6eb8>, '_tf_random_seed': None, '_task_id': 0, '_environment': 'local', '_save_checkpoints_secs': None, '_train_distribute': None, '_keep_checkpoint_max': 20, '_is_chief': True, '_task_type': None, '_protocol': None, '_master': '', 'use_tpu': False, '_save_checkpoints_steps': 2000, '_evaluation_master': '', '_device_fn': None, '_save_summary_steps': 100, '_num_ps_replicas': 0, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'model_file_base1024', 't2t_device_info': {'num_async_rep

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Base learning rate: 2.000000
INFO:tensorflow:Trainable Variables Total size: 64628224
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-04T17:23:37Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_file_base1024/model.ckpt-36000
INFO:tensorflow:Running local_init_op.
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-04T17:31:14Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_file_base1024/model.ckpt-38000
INFO:tensorflow:Running local_init_op.
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-04T17:38:56Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_file_base1024/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensor

## Full length

In [3]:
from tensor2tensor import problems
from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json

MODEL = 'transformer'
TRAIN_DIR = 'model_file_basefull'
DATA_LOC = './data'
PROBLEM = 'onlinereview'

# Define hparams
HPARAMS = 'transformer_base'
hparams = create_hparams(HPARAMS)
# hparams.prepend_mode = 'prepend_inputs_full_attention'
hparams.batch_size = 1024 #2048 #512 
hparams.learning_rate_warmup_steps = 8000
hparams.learning_rate = 0.1 
# hparams.num_encoder_layers = 6
hparams.num_hidden_layers = 6
# hparams.num_decoder_layers = 6
hparams.dropout =0.2
# hparams.num_heads =16
hparams.attention_dropout = 0.2 ##
# hparams.max_length = 0
# hparams.max_input_seq_length = 512

decode_hp = decoding.decode_hparams()
# decode_hp.block_size =6
decode_hp.return_beams = True #True
decode_hp.beam_size = 2

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))


{'learning_rate_cosine_cycle_steps': 250000, 'optimizer_momentum_nesterov': False, 'tpu_enable_host_call': False, 'label_smoothing': 0.1, 'hidden_size': 512, 'nbr_decoder_problems': 1, 'heads_share_relative_embedding': False, 'moe_overhead_eval': 2.0, 'optimizer_multistep_accumulate_steps': None, 'video_num_input_frames': 1, 'prepend_mode': 'none', 'learning_rate_decay_steps': 5000, 'pad_batch': False, 'optimizer_adafactor_beta1': 0.0, 'learning_rate_warmup_steps': 8000, 'split_to_length': 0, 'parameter_attention_key_channels': 0, 'attention_value_channels': 0, 'shared_embedding': False, 'filter_size': 2048, 'shared_embedding_and_softmax_weights': True, 'optimizer_adam_beta2': 0.997, 'multiproblem_fixed_train_length': -1, 'causal_decoder_self_attention': True, 'batch_shuffle_size': 512, 'overload_eval_metric_name': '', 'layer_preprocess_sequence': 'n', 'attention_variables_3d': False, 'unidirectional_encoder': False, 'no_data_parallelism': False, 'summarize_grads': False, 'optimizer': 

In [4]:
import timeit
# running time calculation
start = timeit.default_timer()

t2t_problem = problems.problem(PROBLEM)

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=2000,
    )

t2t_model_base_full = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        problem_name=t2t_problem,
        model_name=MODEL,
        data_dir=DATA_LOC, 
        train_steps=40001,
        eval_steps=200,
        min_eval_frequency=2000,
        eval_use_test_set=True,
    )

# hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_base_full.train_and_evaluate()

# running time check (running on 1GPU server)
stop = timeit.default_timer()
print('Time: {} s'.format(round(stop - start,2)))

Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:Configuring DataParallelism to replicate the model.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_environment': 'local', '_model_dir': 'model_file_basefull', '_eval_distribute': None, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_device_fn': None, '_num_worker_replicas': 0, '_log_step_count_steps': 100, 't2t_device_info': {'num_async_replicas': 1}, '_master': '', '_task_type': None, '_task_id': 0, 'use_tpu': False, '_num_ps_replicas': 0, '_save_checkpoints_steps': 2000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f96c477ab70>, '_train_distribute': None, '_keep_checkpoint_max': 20, '_save_checkpoin

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


INFO:tensorflow:partition: 0 num_data_files: 100
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
INFO:tensorflow:Calling model_fn.


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-05T08:38:50Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_file_basefull/model.ckpt-36000
INFO:tensorflow:Running local_init_op.
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-05T08:47:20Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_file_basefull/model.ckpt-38000
INFO:tensorflow:Running local_init_op.
INFO:tensor

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-05T08:55:52Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_file_basefull/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tensor

## Maxlen 256 -lr=0.05

In [4]:
from tensor2tensor import problems
from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json

MODEL = 'transformer'
TRAIN_DIR = 'model_file_base256_005'
DATA_LOC = './data'
PROBLEM = 'onlinereview'

# Define hparams
HPARAMS = 'transformer_base'
hparams = create_hparams(HPARAMS)
# hparams.prepend_mode = 'prepend_inputs_full_attention'
hparams.batch_size = 1024 #2048 #512 
hparams.learning_rate_warmup_steps = 4000
hparams.learning_rate = 0.05
# hparams.num_encoder_layers = 6
hparams.num_hidden_layers = 6
# hparams.num_decoder_layers = 6
hparams.dropout =0.1
# hparams.num_heads =16
hparams.attention_dropout = 0.1 ##
hparams.max_length = 256
# hparams.max_input_seq_length = 512

decode_hp = decoding.decode_hparams()
# decode_hp.block_size =6
decode_hp.return_beams = True #True
decode_hp.beam_size = 2

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))

t2t_problem = problems.problem(PROBLEM)

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=2000,
    )

t2t_model_base = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        problem_name=t2t_problem,
        model_name=MODEL,
        data_dir=DATA_LOC, 
        train_steps=40001,
        eval_steps=200,
        min_eval_frequency=2000,
        eval_use_test_set=True,
    )

# hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_base.train_and_evaluate()


{'multiproblem_max_target_length': -1, 'use_target_space_embedding': True, 'layer_prepostprocess_dropout_broadcast_dims': '', 'weight_decay': 0.0, 'learning_rate_decay_rate': 1.0, 'multiproblem_mixing_schedule': 'constant', 'optimizer': 'Adam', 'video_num_target_frames': 1, 'vocab_divisor': 1, 'max_input_seq_length': 0, 'optimizer_adafactor_clipping_threshold': 1.0, 'moe_k': 2, 'norm_epsilon': 1e-06, 'scheduled_sampling_gold_mixin_prob': 0.5, 'weight_dtype': 'float32', 'factored_logits': False, 'grad_noise_scale': 0.0, 'learning_rate_schedule': 'constant*linear_warmup*rsqrt_decay*rsqrt_hidden_size', 'learning_rate': 0.05, 'moe_loss_coef': 0.001, 'nbr_decoder_problems': 1, 'min_length_bucket': 8, 'eval_run_autoregressive': False, 'pad_batch': False, 'activation_dtype': 'float32', 'ffn_layer': 'dense_relu_dense', 'initializer_gain': 1.0, 'symbol_modality_skip_top': False, 'layer_preprocess_sequence': 'n', 'sampling_temp': 1.0, 'max_length': 256, 'relu_dropout_broadcast_dims': '', 'moe_nu

/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


INFO:tensorflow:partition: 0 num_data_files: 100


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Setting T2TModel mode to 'train'
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Base learning rate: 2.000000
INFO:tensorflow:Trainable Variables Total size: 64628224
INFO:tensorflow:Non-trainable variables Total size: 5
INFO:tensorflow:Using optimizer Adam


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-06T06:53:20Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_file_base256_005/model.ckpt-34000
INFO:tensorflow:Running local_init_op.
INFO:ten

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-06T07:01:50Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_file_base256_005/model.ckpt-36000
INFO:tensorflow:Running local_init_op.
INFO:ten

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-06T07:10:19Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model_file_base256_005/model.ckpt-38000
INFO:tensorflow:Running local_init_op.
INFO:ten

## Base -512 -0607 try

In [3]:
from tensor2tensor import problems
from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json

MODEL = 'transformer'
TRAIN_DIR = '../model_file_base512'
DATA_LOC = './data'
PROBLEM = 'onlinereview'

# Define hparams
HPARAMS = 'transformer_base'
hparams = create_hparams(HPARAMS)
# hparams.prepend_mode = 'prepend_inputs_full_attention'
hparams.batch_size = 1024 #2048 #2048 #512 
hparams.learning_rate = 0.05 
# hparams.num_encoder_layers = 6
hparams.num_hidden_layers = 6
# hparams.num_decoder_layers = 6
hparams.dropout =0.2
# hparams.num_heads =16
hparams.attention_dropout = 0.2 ##
hparams.max_length = 512
# hparams.max_input_seq_length = 512

decode_hp = decoding.decode_hparams()
decode_hp.return_beams = True
decode_hp.beam_size = 2

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))

t2t_problem = problems.problem(PROBLEM)

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=2000,
    )

t2t_model_base = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        problem_name=t2t_problem,
        model_name=MODEL,
        data_dir=DATA_LOC, 
        train_steps=40201,
        eval_steps=200,
        min_eval_frequency=2000,
        eval_use_test_set=True,
    )

# hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_base.train_and_evaluate()


{'pos': 'timing', 'optimizer_adafactor_beta2': 0.999, 'grad_noise_scale': 0.0, 'activation_dtype': 'float32', 'symbol_modality_skip_top': False, 'shared_embedding': False, 'optimizer_adafactor_multiply_by_parameter_scale': True, 'optimizer_zero_grads': False, 'vocab_divisor': 1, 'summarize_grads': False, 'learning_rate_constant': 2.0, 'warm_start_from_second': '', 'num_encoder_layers': 0, 'multiproblem_reweight_label_loss': False, 'attention_variables_3d': False, 'moe_loss_coef': 0.001, 'optimizer_multistep_accumulate_steps': None, 'use_pad_remover': True, 'split_to_length': 0, 'optimizer_adafactor_decay_type': 'pow', 'optimizer': 'Adam', 'learning_rate_decay_steps': 5000, 'optimizer_adam_beta2': 0.997, 'force_full_predict': False, 'multiproblem_vocab_size': -1, 'attention_key_channels': 0, 'moe_overhead_eval': 2.0, 'heads_share_relative_embedding': False, 'learning_rate_warmup_steps': 8000, 'initializer_gain': 1.0, 'factored_logits': False, 'scheduled_sampling_warmup_steps': 50000, 'n

INFO:tensorflow:Reading data files from ./data/onlinereview-train*


/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


INFO:tensorflow:partition: 0 num_data_files: 100
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-07T10:02:49Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model_file_base512/model.ckpt-36000
INFO:tensorflow:Running local_init_op.
INFO:tens

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-07T10:10:31Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model_file_base512/model.ckpt-38000
INFO:tensorflow:Running local_init_op.
INFO:tens

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-07T10:18:14Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model_file_base512/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:tens

## Base -1024 -0607 try 0.05

In [ ]:
from tensor2tensor import problems
from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json

MODEL = 'transformer'
TRAIN_DIR = '../model_file_base1024'
DATA_LOC = './data'
PROBLEM = 'onlinereview'

# Define hparams
HPARAMS = 'transformer_base'
hparams = create_hparams(HPARAMS)
# hparams.prepend_mode = 'prepend_inputs_full_attention'
hparams.batch_size = 1024 #2048 #2048 #512 
hparams.learning_rate = 0.05 
# hparams.num_encoder_layers = 6
hparams.num_hidden_layers = 6
# hparams.num_decoder_layers = 6
hparams.dropout =0.2
# hparams.num_heads =16
hparams.attention_dropout = 0.2 ##
hparams.max_length = 1024
# hparams.max_input_seq_length = 512

decode_hp = decoding.decode_hparams()
decode_hp.return_beams = True
decode_hp.beam_size = 2

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))

t2t_problem = problems.problem(PROBLEM)

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=2000,
    )

t2t_model_base = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        problem_name=t2t_problem,
        model_name=MODEL,
        data_dir=DATA_LOC, 
        train_steps=40201,
        eval_steps=200,
        min_eval_frequency=2000,
        eval_use_test_set=True,
    )

# hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_base.train_and_evaluate()


{'num_heads': 8, 'learning_rate': 0.05, 'moe_hidden_sizes': '2048', 'multiproblem_mixing_schedule': 'constant', 'moe_loss_coef': 0.001, 'multiproblem_schedule_threshold': 0.5, 'video_num_input_frames': 1, 'split_to_length': 0, 'no_data_parallelism': False, 'moe_overhead_eval': 2.0, 'multiproblem_fixed_train_length': -1, 'multiproblem_max_target_length': -1, 'symbol_modality_num_shards': 16, 'causal_decoder_self_attention': True, 'multiproblem_vocab_size': -1, 'optimizer_adam_beta1': 0.9, 'pretrained_model_dir': '', 'learning_rate_constant': 2.0, 'video_num_target_frames': 1, 'conv_first_kernel': 3, 'grad_noise_scale': 0.0, 'min_length_bucket': 8, 'learning_rate_decay_scheme': 'noam', 'sampling_method': 'argmax', 'max_relative_position': 0, 'multiproblem_schedule_max_examples': 10000000.0, 'moe_k': 2, 'kernel_width': 1, 'initializer_gain': 1.0, 'shared_embedding': False, 'self_attention_type': 'dot_product', 'modality': {}, 'pad_batch': False, 'max_length': 1024, 'clip_grad_norm': 0.0, 

INFO:tensorflow:Reading data files from ./data/onlinereview-train*


/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


INFO:tensorflow:partition: 0 num_data_files: 100
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-07T14:22:46Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model_file_base1024/model.ckpt-36000
INFO:tensorflow:Running local_init_op.
INFO:ten

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-07T14:30:17Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model_file_base1024/model.ckpt-38000
INFO:tensorflow:Running local_init_op.
INFO:ten

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-07T14:37:49Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model_file_base1024/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:ten

## Base -full -0607 try 0.05

In [3]:
from tensor2tensor import problems
from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json

MODEL = 'transformer'
TRAIN_DIR = '../model_file_basefull'
DATA_LOC = './data'
PROBLEM = 'onlinereview'

# Define hparams
HPARAMS = 'transformer_base'
hparams = create_hparams(HPARAMS)
# hparams.prepend_mode = 'prepend_inputs_full_attention'
hparams.batch_size = 1024 #2048 #2048 #512 
hparams.learning_rate = 0.05 
# hparams.num_encoder_layers = 6
hparams.num_hidden_layers = 6
# hparams.num_decoder_layers = 6
hparams.dropout =0.2
# hparams.num_heads =16
hparams.attention_dropout = 0.2 ##
hparams.max_length = 0
# hparams.max_input_seq_length = 512

decode_hp = decoding.decode_hparams()
decode_hp.return_beams = True
decode_hp.beam_size = 2

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))

t2t_problem = problems.problem(PROBLEM)

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=2000,
    )

t2t_model_base = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        problem_name=t2t_problem,
        model_name=MODEL,
        data_dir=DATA_LOC, 
        train_steps=40201,
        eval_steps=200,
        min_eval_frequency=2000,
        eval_use_test_set=True,
    )

# hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_base.train_and_evaluate()


{'weight_dtype': 'float32', 'optimizer_adafactor_beta1': 0.0, 'pretrained_model_dir': '', 'parameter_attention_value_channels': 0, 'use_target_space_embedding': True, 'moe_overhead_train': 1.0, 'layer_prepostprocess_dropout_broadcast_dims': '', 'learning_rate_cosine_cycle_steps': 250000, 'sampling_method': 'argmax', 'modality': {}, 'multiproblem_max_target_length': -1, 'multiproblem_label_weight': 0.5, 'factored_logits': False, 'multiproblem_per_task_threshold': '', 'layer_postprocess_sequence': 'da', 'min_length': 0, 'initializer_gain': 1.0, 'length_bucket_step': 1.1, 'symbol_dropout': 0.0, 'multiproblem_vocab_size': -1, 'initializer': 'uniform_unit_scaling', 'optimizer_momentum_nesterov': False, 'prepend_mode': 'none', 'no_data_parallelism': False, 'hidden_size': 512, 'learning_rate_schedule': 'constant*linear_warmup*rsqrt_decay*rsqrt_hidden_size', 'sampling_temp': 1.0, 'learning_rate_warmup_steps': 8000, 'optimizer': 'Adam', 'use_fixed_batch_size': False, 'daisy_chain_variables': Tr

INFO:tensorflow:Reading data files from ./data/onlinereview-train*
INFO:tensorflow:partition: 0 num_data_files: 100


/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-08T03:27:32Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model_file_basefull/model.ckpt-36000
INFO:tensorflow:Running local_init_op.
INFO:ten

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-08T03:35:04Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model_file_basefull/model.ckpt-38000
INFO:tensorflow:Running local_init_op.
INFO:ten

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-08T03:42:35Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model_file_basefull/model.ckpt-40000
INFO:tensorflow:Running local_init_op.
INFO:ten

## Base -256 -0608 try 0.05

In [3]:
from tensor2tensor import problems
from tensor2tensor import models
from tensor2tensor.utils import registry, trainer_lib, decoding
from tensor2tensor.utils.trainer_lib import create_hparams
from tensor2tensor.utils.trainer_lib import create_hparams, create_run_config, create_experiment
import json

MODEL = 'transformer'
TRAIN_DIR = '../model_file_base_256'
DATA_LOC = './data'
PROBLEM = 'onlinereview'

# Define hparams
HPARAMS = 'transformer_base'
hparams = create_hparams(HPARAMS)
# hparams.prepend_mode = 'prepend_inputs_full_attention'
hparams.batch_size = 1024 #2048 #2048 #512 
hparams.learning_rate = 0.05 
# hparams.num_encoder_layers = 6
hparams.num_hidden_layers = 6
# hparams.num_decoder_layers = 6
hparams.dropout =0.2
# hparams.num_heads =16
hparams.attention_dropout = 0.2 ##
hparams.max_length = 256
# hparams.max_input_seq_length = 512

decode_hp = decoding.decode_hparams()
decode_hp.return_beams = True
decode_hp.beam_size = 2

# print out all Hparams
print(json.loads(hparams.to_json()))
print(json.loads(decode_hp.to_json()))

t2t_problem = problems.problem(PROBLEM)

RUN_CONFIG = create_run_config(
    model_name=MODEL,
    model_dir=TRAIN_DIR,
    save_checkpoints_steps=2000,
    )

t2t_model_base = create_experiment(
        run_config=RUN_CONFIG,
        hparams=hparams,
        decode_hparams=decode_hp,
        problem_name=t2t_problem,
        model_name=MODEL,
        data_dir=DATA_LOC, 
        train_steps=60201,
        eval_steps=200,
        min_eval_frequency=2000,
        eval_use_test_set=True,
    )

# hparams.problem_hparams.was_reversed =False
print('\n hparams.problem_hparams \n',hparams.problem_hparams)

# training
t2t_model_base.train_and_evaluate()


{'mlperf_mode': False, 'parameter_attention_value_channels': 0, 'max_length': 256, 'sampling_method': 'argmax', 'use_target_space_embedding': True, 'conv_first_kernel': 3, 'weight_decay': 0.0, 'moe_hidden_sizes': '2048', 'max_relative_position': 0, 'weight_dtype': 'float32', 'num_hidden_layers': 6, 'learning_rate_constant': 2.0, 'multiproblem_label_weight': 0.5, 'hidden_size': 512, 'kernel_width': 1, 'multiproblem_vocab_size': -1, 'optimizer_multistep_accumulate_steps': None, 'label_smoothing': 0.1, 'use_fixed_batch_size': False, 'summarize_grads': False, 'pretrained_model_dir': '', 'multiproblem_schedule_max_examples': 10000000.0, 'proximity_bias': False, 'eval_run_autoregressive': False, 'symbol_dropout': 0.0, 'activation_dtype': 'float32', 'prepend_mode': 'none', 'norm_epsilon': 1e-06, 'kernel_height': 3, 'moe_k': 2, 'vocab_divisor': 1, 'relu_dropout_broadcast_dims': '', 'learning_rate_schedule': 'constant*linear_warmup*rsqrt_decay*rsqrt_hidden_size', 'split_to_length': 0, 'num_deco

INFO:tensorflow:Reading data files from ./data/onlinereview-train*
INFO:tensorflow:partition: 0 num_data_files: 100


/usr/local/lib/python3.5/dist-packages/tensorflow/python/platform/tf_logging.py:161: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  get_logger().warn(msg, *args, **kwargs)


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)
/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


...

INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-08T09:24:56Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model_file_base_256/model.ckpt-56000
INFO:tensorflow:Running local_init_op.
INFO:ten

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-08T09:33:24Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model_file_base_256/model.ckpt-58000
INFO:tensorflow:Running local_init_op.
INFO:ten

/usr/local/lib/python3.5/dist-packages/numpy/lib/type_check.py:546: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  'a.item() instead', DeprecationWarning, stacklevel=1)


INFO:tensorflow:Setting T2TModel mode to 'eval'
INFO:tensorflow:Setting hparams.label_smoothing to 0.0
INFO:tensorflow:Setting hparams.symbol_dropout to 0.0
INFO:tensorflow:Setting hparams.dropout to 0.0
INFO:tensorflow:Setting hparams.relu_dropout to 0.0
INFO:tensorflow:Setting hparams.attention_dropout to 0.0
INFO:tensorflow:Setting hparams.layer_prepostprocess_dropout to 0.0
INFO:tensorflow:Using variable initializer: uniform_unit_scaling
INFO:tensorflow:Transforming feature 'inputs' with symbol_modality_40055_512.bottom
INFO:tensorflow:Transforming feature 'targets' with symbol_modality_40055_512.targets_bottom
INFO:tensorflow:Building model body
INFO:tensorflow:Transforming body output with symbol_modality_40055_512.top
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-08T09:41:53Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../model_file_base_256/model.ckpt-60000
INFO:tensorflow:Running local_init_op.
INFO:ten